In [70]:
import pandas as pd
import os
import numpy as np

In [71]:
os.listdir('../data')

['BGC2line.pkl',
 'BGC_labels_dataset.csv',
 'BGC_labels_dataset.pkl',
 'BGC_labels_dataset_positive.csv',
 'BGC_labels_eval_dataset.csv',
 'getNegativeSamples.py',
 'mibig_labels.pkl',
 'positiveSamples.py',
 'sampling_chi2.py',
 'Sampling_chi2_2.png']

In [72]:
BGC_labels_dataset = pd.read_csv('../data/BGC_labels_dataset.csv')
BGC_labels_dataset

,ID,sentence,labels,isBGC
0,BGC0000436,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes
1,BGC0000918,Cluster_0698230 EC:1.13.11.27 Cluster_1230425 ...,Other,Yes
2,BGC0000740,Cluster_1607346 Cluster_0676609 Cluster_090172...,Saccharide,Yes
3,BGC0002659,Cluster_1042555 Cluster_0095569 Cluster_176389...,RiPP,Yes
4,BGC0001750,Cluster_0175250 Cluster_1313057 EC:6.2.1.3 Clu...,NRP Polyketide,Yes
...,...,...,...,...
10003,HalfMibigHalfGenome_7501,EC:5.3.3.2 Cluster_1049777 Cluster_1641406 Clu...,Non,No
10004,HalfMibigHalfGenome_7502,Cluster_1322604 EC:6.2.1.3 Cluster_0864922 EC:...,Non,No
10005,HalfMibigHalfGenome_7503,Cluster_0513799 Cluster_1249665 Cluster_1682136,Non,No
10006,HalfMibigHalfGenome_7504,Cluster_1843822 Cluster_0940597 Cluster_032120...,Non,No


In [73]:
BGC_labels_dataset_positive = BGC_labels_dataset[BGC_labels_dataset['isBGC']=='Yes']
BGC_labels_dataset_positive

,ID,sentence,labels,isBGC
0,BGC0000436,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes
1,BGC0000918,Cluster_0698230 EC:1.13.11.27 Cluster_1230425 ...,Other,Yes
2,BGC0000740,Cluster_1607346 Cluster_0676609 Cluster_090172...,Saccharide,Yes
3,BGC0002659,Cluster_1042555 Cluster_0095569 Cluster_176389...,RiPP,Yes
4,BGC0001750,Cluster_0175250 Cluster_1313057 EC:6.2.1.3 Clu...,NRP Polyketide,Yes
...,...,...,...,...
2497,BGC0000177,Cluster_1377462 Cluster_0008689 Cluster_062041...,Polyketide,Yes
2498,BGC0000892,Cluster_0385013 Cluster_1806792 Cluster_045151...,Other,Yes
2499,BGC0001718,EC:2.7.7.48 Cluster_1722574 Cluster_0431146 Cl...,NRP,Yes
2500,BGC0000213,Cluster_0417466 Cluster_1653936 Cluster_156762...,Polyketide,Yes


In [74]:
BGC_labels_dataset_negative = BGC_labels_dataset[BGC_labels_dataset['isBGC']=='No']
BGC_labels_dataset_negative

,ID,sentence,labels,isBGC
2502,HalfMibigHalfGenome_0,Cluster_1760951 Cluster_1044407 Cluster_043984...,Non,No
2503,HalfMibigHalfGenome_1,Cluster_1786836 EC:4.2.3.51 Cluster_0057570 Cl...,Non,No
2504,HalfMibigHalfGenome_2,Cluster_0010015 Cluster_1400143 EC:2.4.1.280 C...,Non,No
2505,HalfMibigHalfGenome_3,Cluster_0077106 EC:4.1.2.13 Cluster_0831664 Cl...,Non,No
2506,HalfMibigHalfGenome_4,Cluster_0410901 EC:2.7.1.31 Unknown EC:1.1.1.1...,Non,No
...,...,...,...,...
10003,HalfMibigHalfGenome_7501,EC:5.3.3.2 Cluster_1049777 Cluster_1641406 Clu...,Non,No
10004,HalfMibigHalfGenome_7502,Cluster_1322604 EC:6.2.1.3 Cluster_0864922 EC:...,Non,No
10005,HalfMibigHalfGenome_7503,Cluster_0513799 Cluster_1249665 Cluster_1682136,Non,No
10006,HalfMibigHalfGenome_7504,Cluster_1843822 Cluster_0940597 Cluster_032120...,Non,No


In [75]:
import random

In [182]:
def generateInsertList(num, maxLen=64, notInsertProb:float=0.5, maxNonBGCgenes:int=8):
    '''
    生成插入到序列每个空位非BGC的基因个数，首尾也插入，有插入概率与长度的限制
    num: BGC序列长度
    notInsertPro: 该空位不插入非BGC基因的概率，此处为概率而非比例，有可能每个空位都插入也有可能每个空位均不插入
    maxNonBGCgenes: 一个空位插入非BGC基因的最大长度
    '''
    randomNonBGClen = list(range(0, maxNonBGCgenes+1))
    randomNonBGClenWeight = [notInsertProb, *[(1-notInsertProb)/maxNonBGCgenes]*maxNonBGCgenes]
    randomLen = random.choices(population=randomNonBGClen, weights=randomNonBGClenWeight, k=num-1)
    randomStart, randomEnd = [0], [0]
    leftLen = maxLen-num-sum(randomLen)
    randomStart = [0]
    randomEnd = [0]
    if leftLen>0:
        randomStart = random.randint(0, leftLen)
        randomEnd = [leftLen - randomStart]
        randomStart = [randomStart]
    return randomStart + randomLen + randomEnd
a = generateInsertList(20, 64, 0.8, 8)
print(a)


[9, 0, 0, 4, 1, 0, 0, 2, 0, 0, 0, 6, 0, 0, 5, 0, 0, 4, 0, 0, 13]


In [113]:
import random

def random_num_with_fix_total(maxValue, num):
    '''生成总和固定的整数序列
    maxvalue: 序列总和
    num：要生成的整数个数'''
    a = random.choices(range(0,maxValue), k=num-1) # 在1~99之间，采集20个数据
    a.append(0)   # 加上数据开头
    a.append(maxValue)
    a = sorted(a)
    b = [ a[i]-a[i-1] for i in range(1, len(a)) ] # 列表推导式，计算列表中每两个数之间的间隔
    print(b)
    return b 
    # print(len(b))
    # print( sum(b))

random_num_with_fix_total(64-20, 4)


[8, 9, 3, 24]


[8, 9, 3, 24]

In [112]:
def generateInsertListRatio(num, maxLen, notInsertRatio=0.6):
    '''
    生成插入到序列每个空位非BGC的基因个数，有固定比例的空位不被插入
    num：BGC序列长度
    notInsertRatio：固定比例的空位不被插入
    '''
    randomPlace = [0]+ random.sample(population=list(range(1, num)), k=int(num*(1-notInsertRatio))) +[num]
    print(randomPlace)
    randomLen = []
    leftLen = maxLen - num
    randomPlaceLen = random_num_with_fix_total(maxValue=leftLen, num=len(randomPlace))
    j = 0
    for i in range(num+1):
        if i in randomPlace:
            randomLen.append(randomPlaceLen[j])
            j+=1
        else:
            randomLen.append(0)
        # print(randomLen)
    return randomLen
b = generateInsertListRatio(20, 64, 0.8)
b
# sum(b)


[0, 9, 16, 6, 20]
[9, 3, 12, 13, 7]


[9, 0, 0, 0, 0, 0, 3, 0, 0, 12, 0, 0, 0, 0, 0, 0, 13, 0, 0, 0, 7]

In [115]:
import pickle

In [117]:
with open('../data/word_frequency_dic_min3.pkl', 'rb') as fp:
    word_frequency_dict_min3 = pickle.load(fp)

with open('../data/mibig_word_frequency_dic.pkl', 'rb') as fp:
    mibig_word_frequency_dict = pickle.load(fp)

word_frequency_dict_min3

{'EC:1.6.5.11': 221817,
 'Cluster_0198348': 1186,
 'Cluster_0762898': 35261,
 'Cluster_0300183': 28522,
 'Cluster_1321460': 1537,
 'Cluster_0394599': 981,
 'Cluster_1805401': 30541,
 'Cluster_1866529': 26965,
 'Cluster_0289967': 24900,
 'Cluster_0561280': 31147,
 'Cluster_1583857': 37753,
 'Cluster_1639562': 31908,
 'Cluster_0608304': 31285,
 'Cluster_0409832': 25218,
 'Cluster_0840679': 31357,
 'Cluster_1038875': 31263,
 'Cluster_0352804': 7019,
 'Cluster_1552942': 31201,
 'Cluster_0988836': 31669,
 'Cluster_0314129': 30857,
 'Cluster_0345707': 30568,
 'Cluster_0492924': 2196,
 'Cluster_0673339': 1497,
 'Cluster_0453759': 1751,
 'Cluster_1328707': 32436,
 'Cluster_0652876': 34328,
 'Cluster_0480316': 28201,
 'Cluster_1680973': 747,
 'Cluster_0970700': 265201,
 'Cluster_1119276': 139,
 'Cluster_0876677': 10413,
 'Cluster_1755588': 479,
 'Cluster_0317541': 241,
 'EC:2.4.99.17': 26962,
 'Cluster_0912295': 1107,
 'Cluster_0901791': 32517,
 'Cluster_1274850': 30479,
 'EC:4.1.2.13': 45328,


In [121]:

class RandomGenerator:
    def __init__(self, samplingWeights, id2word) -> None:
        assert len(samplingWeights)==len(id2word)
        self.population = list(range(0, len(samplingWeights)))
        self.samplingWeights = samplingWeights
        self.candidates = []
        self.i = 0

    def draw(self):
        if self.i == len(self.candidates):
            self.candidates = np.random.choice(self.population, int(1e8), p=self.samplingWeights)
            self.i = 0
        self.i +=1
        return self.candidates[self.i-1]
            

In [176]:
def getOneSeq(seq, maxLen, generator, id2word, notInsertProb=0.8, maxNonBGCgenes=5):
    num = len(seq)
    insertList = generateInsertList(num=num, maxLen=maxLen, notInsertProb=notInsertProb, maxNonBGCgenes=maxNonBGCgenes)
    # print(insertList)
    trainSeq = []
    trainSeqLabels = []
    for i in range(len(insertList)):
        randomNonBGCList = []
        while(len(randomNonBGCList)<insertList[i]):
            a = generator.draw()
            if a not in seq:
                randomNonBGCList.append(a)
        # print(randomNonBGCList)
        trainSeq += [id2word[j] for j in randomNonBGCList]
        trainSeqLabels += [0]*insertList[i]
        if i<num:
            trainSeq += [seq[i]]
            trainSeqLabels += [1]
    return trainSeq, trainSeqLabels

        


In [119]:
with open('../data/gensim_id2word.pkl', 'rb')as fp:
    id2word = pickle.load(fp)
id2word

['EC:2.7.13.3',
 'EC:3.6.4.12',
 'Cluster_0970700',
 'EC:3.6.4.13',
 'EC:1.6.5.11',
 'EC:3.6.3.28',
 'EC:2.7.1.191',
 'EC:3.6.3.17',
 'Cluster_1433175',
 'EC:1.6.5.3',
 'EC:2.7.7.6',
 'EC:3.6.3.41',
 'EC:1.1.1.100',
 'Cluster_0554161',
 'Cluster_0375649',
 'Cluster_1845293',
 'Cluster_0231832',
 'Cluster_1192257',
 'Cluster_0447434',
 'EC:5.2.1.8',
 'Cluster_0074328',
 'EC:2.7.7.7',
 'Cluster_1474940',
 'EC:3.6.3.24',
 'Cluster_0304335',
 'EC:5.99.1.3',
 'EC:6.2.1.3',
 'Cluster_1449151',
 'Cluster_1606065',
 'EC:1.9.3.1',
 'Cluster_0410901',
 'Cluster_0929395',
 'Cluster_0659041',
 'Cluster_0103885',
 'Cluster_0968728',
 'Cluster_0970273',
 'EC:3.6.3.14',
 'Cluster_0571584',
 'Cluster_1140527',
 'EC:1.11.1.15',
 'EC:3.1.26.5',
 'Cluster_0866292',
 'Cluster_0519424',
 'Cluster_0684496',
 'EC:2.4.1.129',
 'Cluster_0402868',
 'Cluster_0750400',
 'EC:2.7.11.1',
 'Cluster_0385013',
 'Cluster_0201874',
 'Cluster_1340063',
 'Cluster_0831169',
 'EC:3.6.3.25',
 'Cluster_1838152',
 'Cluster_1313

In [ ]:
samplingWeights = np.array([word_frequency_dict_min3[word] for word in id2word])
samplingWeights = samplingWeights/sum(samplingWeights)
generator = RandomGenerator(samplingWeights, id2word=id2word)



In [133]:
generator.draw()

78925

In [147]:
trainSeq, trainSeqLabel = getOneSeq(['Cluster_1153963', 'Cluster_1501678', 'Cluster_1607345', 'Cluster_1534836', 'Cluster_1124421', 'EC:6.2.1.3', 'Cluster_0034481'], 
          maxLen=64, generator=generator, id2word=id2word)

[27, 0, 0, 0, 0, 5, 0, 25]
[907, 2090, 817, 244, 12332, 3657, 11989, 22265, 1159, 3178, 20003, 3649, 524, 1379, 1377, 22, 277, 16827, 1, 1434, 91904, 10, 18750, 191999, 12446, 4986, 2588]
[]
[]
[]
[]
[4245, 10010, 415, 967, 3109]
[]
[9390, 51484, 1196, 4607, 25, 754, 338437, 232, 2515, 37016, 69, 11128, 393, 1348, 9305, 41846, 367, 1958, 7568, 5037, 4187, 299873, 3881, 7051, 10878]


In [149]:
print(trainSeq)
print(trainSeqLabel)

['Cluster_1125707', 'EC:5.4.99.22', 'Cluster_1412854', 'EC:3.5.1.28', 'Cluster_0457612', 'Cluster_0127760', 'Cluster_1049777', 'Cluster_0267278', 'Cluster_0901555', 'Cluster_0476615', 'Cluster_0406898', 'EC:5.4.99.20', 'Cluster_1728431', 'Cluster_1562238', 'Cluster_0470094', 'Cluster_1474940', 'EC:1.7.99.4', 'Cluster_0166426', 'EC:3.6.4.12', 'EC:2.6.1.85', 'Cluster_1227261', 'EC:2.7.7.6', 'Cluster_1375676', 'Cluster_1659629', 'Cluster_0800968', 'Cluster_1025026', 'EC:3.4.11.23', 'Cluster_1153963', 'Cluster_1501678', 'Cluster_1607345', 'Cluster_1534836', 'Cluster_1124421', 'Cluster_1196643', 'Cluster_0163090', 'EC:2.5.1.75', 'Cluster_0850761', 'Cluster_0066125', 'EC:6.2.1.3', 'Cluster_0034481', 'Cluster_0043228', 'Cluster_0090990', 'Cluster_0664345', 'Cluster_0532087', 'EC:5.99.1.3', 'Cluster_1657215', 'Cluster_1141989', 'Cluster_1810480', 'Cluster_1222735', 'Cluster_1419921', 'EC:6.1.1.20', 'Cluster_0808976', 'Cluster_1862059', 'Cluster_1362396', 'Cluster_1607079', 'Cluster_1689625', '

In [151]:
print(len(trainSeq), len(trainSeqLabel))


64 64


In [152]:
len(word_frequency_dict_min3), len(id2word)

(640854, 640854)

In [156]:
BGC_labels_dataset_positive['sentence']

0       [Cluster_1153963, Cluster_1501678, Cluster_160...
1       [Cluster_0698230, EC:1.13.11.27, Cluster_12304...
2       [Cluster_1607346, Cluster_0676609, Cluster_090...
3       [Cluster_1042555, Cluster_0095569, Cluster_176...
4       [Cluster_0175250, Cluster_1313057, EC:6.2.1.3,...
                              ...                        
2497    [Cluster_1377462, Cluster_0008689, Cluster_062...
2498    [Cluster_0385013, Cluster_1806792, Cluster_045...
2499    [EC:2.7.7.48, Cluster_1722574, Cluster_0431146...
2500    [Cluster_0417466, Cluster_1653936, Cluster_156...
2501    [Cluster_0923528, Cluster_1221717, Cluster_030...
Name: sentence, Length: 2502, dtype: object

In [166]:
BGC_labels_dataset_positive

,ID,sentence,labels,isBGC
0,BGC0000436,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes
1,BGC0000918,"[Cluster_0698230, EC:1.13.11.27, Cluster_12304...",Other,Yes
2,BGC0000740,"[Cluster_1607346, Cluster_0676609, Cluster_090...",Saccharide,Yes
3,BGC0002659,"[Cluster_1042555, Cluster_0095569, Cluster_176...",RiPP,Yes
4,BGC0001750,"[Cluster_0175250, Cluster_1313057, EC:6.2.1.3,...",NRP Polyketide,Yes
...,...,...,...,...
2497,BGC0000177,"[Cluster_1377462, Cluster_0008689, Cluster_062...",Polyketide,Yes
2498,BGC0000892,"[Cluster_0385013, Cluster_1806792, Cluster_045...",Other,Yes
2499,BGC0001718,"[EC:2.7.7.48, Cluster_1722574, Cluster_0431146...",NRP,Yes
2500,BGC0000213,"[Cluster_0417466, Cluster_1653936, Cluster_156...",Polyketide,Yes


In [175]:
BGC_TD_dataset_positive = pd.DataFrame(columns=['ID', 'sentence', 'labels', 'isBGC', 'TDsentence', 'TDlabels'])
BGC_TD_dataset_positive.loc[0]=[1,1,1,1,1,1]
BGC_TD_dataset_positive

,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,1,1,1,1,1,1


In [197]:
random.seed(42)
def generatePostiveSamples(dataset, maxLen, notInsertPorb, maxNonBGCgenes):
    generator = RandomGenerator(samplingWeights=samplingWeights, id2word=id2word)
    BGC_TD_dataset_positive = pd.DataFrame(columns=['ID', 'sentence', 'labels', 'isBGC', 'TDsentence', 'TDlabels'])
    for i in range(len(dataset)):
        for j in range(5):
            trainSeq, trainSeqLabel = getOneSeq(dataset.iloc[i]['sentence'], maxLen=maxLen, generator=generator, 
                                                id2word=id2word, notInsertProb=notInsertPorb, maxNonBGCgenes=maxNonBGCgenes)
            trainSeq = trainSeq[:maxLen]
            trainSeqLabel = trainSeqLabel[:maxLen]
            BGC_TD_dataset_positive.loc[len(BGC_TD_dataset_positive)] = [dataset.iloc[i]['ID']+f'_{j}', 
                                                                          dataset.iloc[i]['sentence'], dataset.iloc[i]['labels'], dataset.iloc[i]['isBGC'], 
                                                                          trainSeq, trainSeqLabel]
    return BGC_TD_dataset_positive


In [198]:
BGC_TD_dataset_positive = generatePostiveSamples(BGC_labels_dataset_positive, maxLen=128,notInsertPorb=0.8, maxNonBGCgenes=5)
BGC_TD_dataset_positive

,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,BGC0000436_0,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[EC:3.1.1.23, EC:1.2.1.12, Cluster_0826651, Cl...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,BGC0000436_1,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_0121728, Cluster_1707520, Cluster_068...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,BGC0000436_2,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1153963, Cluster_1501678, Cluster_160...","[1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,BGC0000436_3,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_0636113, EC:4.1.1.17, Cluster_1359171...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
4,BGC0000436_4,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_0898720, Cluster_0948693, Cluster_150...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
12505,BGC0001595_0,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[EC:2.7.4.16, Cluster_0087115, Cluster_0059320...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12506,BGC0001595_1,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[EC:3.5.3.18, Cluster_0715334, Cluster_0133357...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12507,BGC0001595_2,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_1747296, Cluster_0169359, Cluster_036...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12508,BGC0001595_3,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[EC:3.5.1.88, Cluster_0329918, EC:1.2.4.2, Clu...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [199]:
BGC_TD_dataset_positive['length'] = BGC_TD_dataset_positive['sentence'].apply(lambda x:len(x))
BGC_TD_dataset_positive['TDlength'] = BGC_TD_dataset_positive['TDsentence'].apply(lambda x:len(x))
BGC_TD_dataset_positive

,ID,sentence,labels,isBGC,TDsentence,TDlabels,length,TDlength
0,BGC0000436_0,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[EC:3.1.1.23, EC:1.2.1.12, Cluster_0826651, Cl...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7,128
1,BGC0000436_1,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_0121728, Cluster_1707520, Cluster_068...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7,128
2,BGC0000436_2,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1153963, Cluster_1501678, Cluster_160...","[1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",7,128
3,BGC0000436_3,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_0636113, EC:4.1.1.17, Cluster_1359171...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...",7,128
4,BGC0000436_4,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_0898720, Cluster_0948693, Cluster_150...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7,128
...,...,...,...,...,...,...,...,...
12505,BGC0001595_0,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[EC:2.7.4.16, Cluster_0087115, Cluster_0059320...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",23,128
12506,BGC0001595_1,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[EC:3.5.3.18, Cluster_0715334, Cluster_0133357...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",23,128
12507,BGC0001595_2,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_1747296, Cluster_0169359, Cluster_036...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",23,128
12508,BGC0001595_3,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[EC:3.5.1.88, Cluster_0329918, EC:1.2.4.2, Clu...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",23,128


In [200]:
print(max(BGC_TD_dataset_positive['length']),
max(BGC_TD_dataset_positive['TDlength']))

115 128


In [207]:
BGC_TD_dataset_positive

,ID,sentence,labels,isBGC,TDsentence,TDlabels,length,TDlength
0,BGC0000436_0,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[EC:3.1.1.23, EC:1.2.1.12, Cluster_0826651, Cl...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7,128
1,BGC0000436_1,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_0121728, Cluster_1707520, Cluster_068...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7,128
2,BGC0000436_2,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1153963, Cluster_1501678, Cluster_160...","[1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",7,128
3,BGC0000436_3,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_0636113, EC:4.1.1.17, Cluster_1359171...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...",7,128
4,BGC0000436_4,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_0898720, Cluster_0948693, Cluster_150...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7,128
...,...,...,...,...,...,...,...,...
12505,BGC0001595_0,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[EC:2.7.4.16, Cluster_0087115, Cluster_0059320...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",23,128
12506,BGC0001595_1,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[EC:3.5.3.18, Cluster_0715334, Cluster_0133357...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",23,128
12507,BGC0001595_2,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_1747296, Cluster_0169359, Cluster_036...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",23,128
12508,BGC0001595_3,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[EC:3.5.1.88, Cluster_0329918, EC:1.2.4.2, Clu...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",23,128


In [209]:
BGC_TD_dataset_positive['sentence'] = BGC_TD_dataset_positive['sentence'].apply(lambda x:' '.join(x))
BGC_TD_dataset_positive['TDsentence'] = BGC_TD_dataset_positive['TDsentence'].apply(lambda x:' '.join(x))
BGC_TD_dataset_positive['TDlabels'] = BGC_TD_dataset_positive['TDlabels'].apply(lambda x:[str(i) for i in x]).apply(lambda x:' '.join(x))
BGC_TD_dataset_positive

,ID,sentence,labels,isBGC,TDsentence,TDlabels,length,TDlength
0,BGC0000436_0,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,EC:3.1.1.23 EC:1.2.1.12 Cluster_0826651 Cluste...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,7,128
1,BGC0000436_1,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_0121728 Cluster_1707520 Cluster_068090...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,7,128
2,BGC0000436_2,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_1153963 Cluster_1501678 Cluster_160734...,1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,7,128
3,BGC0000436_3,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_0636113 EC:4.1.1.17 Cluster_1359171 EC...,0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 ...,7,128
4,BGC0000436_4,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_0898720 Cluster_0948693 Cluster_150030...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,7,128
...,...,...,...,...,...,...,...,...
12505,BGC0001595_0,Cluster_0923528 Cluster_1221717 Cluster_030676...,Alkaloid,Yes,EC:2.7.4.16 Cluster_0087115 Cluster_0059320 Cl...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,23,128
12506,BGC0001595_1,Cluster_0923528 Cluster_1221717 Cluster_030676...,Alkaloid,Yes,EC:3.5.3.18 Cluster_0715334 Cluster_0133357 EC...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,23,128
12507,BGC0001595_2,Cluster_0923528 Cluster_1221717 Cluster_030676...,Alkaloid,Yes,Cluster_1747296 Cluster_0169359 Cluster_036396...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,23,128
12508,BGC0001595_3,Cluster_0923528 Cluster_1221717 Cluster_030676...,Alkaloid,Yes,EC:3.5.1.88 Cluster_0329918 EC:1.2.4.2 Cluster...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,23,128


In [211]:
BGC_TD_dataset_positive.to_csv('../data/BGC_TD_dataset_positive.csv')

In [212]:
word_frequency_dict_min3

{'EC:1.6.5.11': 221817,
 'Cluster_0198348': 1186,
 'Cluster_0762898': 35261,
 'Cluster_0300183': 28522,
 'Cluster_1321460': 1537,
 'Cluster_0394599': 981,
 'Cluster_1805401': 30541,
 'Cluster_1866529': 26965,
 'Cluster_0289967': 24900,
 'Cluster_0561280': 31147,
 'Cluster_1583857': 37753,
 'Cluster_1639562': 31908,
 'Cluster_0608304': 31285,
 'Cluster_0409832': 25218,
 'Cluster_0840679': 31357,
 'Cluster_1038875': 31263,
 'Cluster_0352804': 7019,
 'Cluster_1552942': 31201,
 'Cluster_0988836': 31669,
 'Cluster_0314129': 30857,
 'Cluster_0345707': 30568,
 'Cluster_0492924': 2196,
 'Cluster_0673339': 1497,
 'Cluster_0453759': 1751,
 'Cluster_1328707': 32436,
 'Cluster_0652876': 34328,
 'Cluster_0480316': 28201,
 'Cluster_1680973': 747,
 'Cluster_0970700': 265201,
 'Cluster_1119276': 139,
 'Cluster_0876677': 10413,
 'Cluster_1755588': 479,
 'Cluster_0317541': 241,
 'EC:2.4.99.17': 26962,
 'Cluster_0912295': 1107,
 'Cluster_0901791': 32517,
 'Cluster_1274850': 30479,
 'EC:4.1.2.13': 45328,


In [215]:
max(mibig_word_frequency_dict.values())

761

In [220]:
not_mibig_word_frequency_dict_list = list(set(word_frequency_dict_min3.keys()) - set(mibig_word_frequency_dict.keys()))
not_mibig_word_frequency_dict_list

['Cluster_1294691',
 'Cluster_0837238',
 'Cluster_1239348',
 'Cluster_0655786',
 'Cluster_0711829',
 'Cluster_1174086',
 'Cluster_1654920',
 'Cluster_1850614',
 'Cluster_0272036',
 'Cluster_0431910',
 'Cluster_1693219',
 'Cluster_0900491',
 'Cluster_0516260',
 'Cluster_1451027',
 'Cluster_1735654',
 'Cluster_0650980',
 'Cluster_1008825',
 'Cluster_1703089',
 'Cluster_0856220',
 'Cluster_0299680',
 'Cluster_0007850',
 'Cluster_0212217',
 'Cluster_1126767',
 'Cluster_0036874',
 'Cluster_0006160',
 'Cluster_0117539',
 'Cluster_1559870',
 'Cluster_1731015',
 'Cluster_0444359',
 'Cluster_0591276',
 'Cluster_0037294',
 'Cluster_0226014',
 'Cluster_1033857',
 'Cluster_0300509',
 'Cluster_1365902',
 'Cluster_1504358',
 'Cluster_0430503',
 'Cluster_1107507',
 'Cluster_1913520',
 'Cluster_0823501',
 'Cluster_0563890',
 'Cluster_0198099',
 'Cluster_1812924',
 'Cluster_0862949',
 'Cluster_1651106',
 'Cluster_0758645',
 'Cluster_1061271',
 'Cluster_0650676',
 'Cluster_1290133',
 'Cluster_0903995',


In [222]:
not_mibig_word_frequency_dict = dict()
for id in not_mibig_word_frequency_dict_list:
    not_mibig_word_frequency_dict[id] = word_frequency_dict_min3[id]
not_mibig_word_frequency_dict    

{'Cluster_1294691': 3,
 'Cluster_0837238': 7,
 'Cluster_1239348': 579,
 'Cluster_0655786': 11,
 'Cluster_0711829': 3,
 'Cluster_1174086': 10,
 'Cluster_1654920': 16,
 'Cluster_1850614': 22,
 'Cluster_0272036': 3,
 'Cluster_0431910': 95,
 'Cluster_1693219': 6,
 'Cluster_0900491': 7,
 'Cluster_0516260': 4,
 'Cluster_1451027': 3,
 'Cluster_1735654': 12,
 'Cluster_0650980': 22,
 'Cluster_1008825': 7,
 'Cluster_1703089': 47,
 'Cluster_0856220': 41,
 'Cluster_0299680': 22,
 'Cluster_0007850': 3,
 'Cluster_0212217': 158,
 'Cluster_1126767': 27,
 'Cluster_0036874': 3,
 'Cluster_0006160': 43,
 'Cluster_0117539': 3,
 'Cluster_1559870': 3,
 'Cluster_1731015': 18,
 'Cluster_0444359': 131,
 'Cluster_0591276': 7,
 'Cluster_0037294': 4,
 'Cluster_0226014': 4,
 'Cluster_1033857': 9,
 'Cluster_0300509': 133,
 'Cluster_1365902': 5,
 'Cluster_1504358': 15,
 'Cluster_0430503': 38,
 'Cluster_1107507': 6,
 'Cluster_1913520': 3,
 'Cluster_0823501': 4,
 'Cluster_0563890': 54,
 'Cluster_0198099': 5,
 'Cluster_

In [224]:
halfMibigSamplingWeights = []
ratio = 0.5
sumMibigFrequency = sum(mibig_word_frequency_dict.values())
sumNotMibigFrequency = sum(not_mibig_word_frequency_dict.values())
for i in range(len(id2word)):
    if id2word[i] in mibig_word_frequency_dict:
        halfMibigSamplingWeights.append(mibig_word_frequency_dict[id2word[i]]/sumMibigFrequency*ratio)
    elif id2word[i] in not_mibig_word_frequency_dict:
        halfMibigSamplingWeights.append(not_mibig_word_frequency_dict[id2word[i]]/sumNotMibigFrequency*ratio)
    else:
        raise
halfMibigSamplingWeights = np.array(halfMibigSamplingWeights)
halfMibigSamplingWeights

array([1.04944599e-03, 6.71157319e-04, 9.15214526e-04, ...,
       2.06520997e-08, 2.06520997e-08, 3.66085811e-05])

In [226]:
len(halfMibigSamplingWeights)

640854

In [227]:

from scipy.stats import chi2
df = 2.7008291842129593  # 自由度参数
loc = 0.7619598528358154  # 位置参数
scale = 5.7855219372765525  # 缩放参数
#按照卡方分布长度随机采样
sentences_length = np.round(chi2.rvs(df, loc=loc, scale=scale, size=len(BGC_labels_dataset_positive))).astype(int)
NegativeDataset = []
for i in range(len(sentences_length)):
    NegativeDataset.append(random.choices(population=id2word, weights=halfMibigSamplingWeights, k=sentences_length[i]))

random.seed(43)
Ngenerator = RandomGenerator(samplingWeights=samplingWeights, id2word=id2word)
BGC_TD_dataset_negative = pd.DataFrame(columns=['ID', 'sentence', 'labels', 'isBGC', 'TDsentence', 'TDlabels'])
for i in range(len(NegativeDataset)):
    for j in range(5):
        trainSeq, trainSeqLabel = getOneSeq(NegativeDataset[i], maxLen=128, generator=Ngenerator, id2word=id2word, notInsertProb=0.8, maxNonBGCgenes=5)
        trainSeq = trainSeq[:128]
        trainSeqLabel = trainSeqLabel[:128]
        BGC_TD_dataset_negative.loc[len(BGC_TD_dataset_negative)] = [f'NonBGC_{i}_{j}', NegativeDataset[i], 'Non', 'No', 
                                                                    trainSeq, trainSeqLabel]


In [228]:
BGC_TD_dataset_negative

,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,NonBGC_0_0,"[Cluster_1645774, Cluster_1021965, Cluster_186...",Non,No,"[Cluster_0510929, EC:1.1.1.22, EC:2.7.7.19, Cl...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ..."
1,NonBGC_0_1,"[Cluster_1645774, Cluster_1021965, Cluster_186...",Non,No,"[Cluster_1037973, EC:3.6.3.25, EC:3.2.1.177, C...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, ..."
2,NonBGC_0_2,"[Cluster_1645774, Cluster_1021965, Cluster_186...",Non,No,"[Cluster_0364863, EC:2.7.1.180, Cluster_034784...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,NonBGC_0_3,"[Cluster_1645774, Cluster_1021965, Cluster_186...",Non,No,"[Cluster_0525603, EC:1.1.1.85, Cluster_1645774...","[0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, ..."
4,NonBGC_0_4,"[Cluster_1645774, Cluster_1021965, Cluster_186...",Non,No,"[EC:1.2.4.2, Cluster_0945303, Cluster_1541072,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
12505,NonBGC_2501_0,"[Cluster_0992319, Cluster_0120706, Cluster_008...",Non,No,"[EC:3.5.2.2, Cluster_0279200, Cluster_1308801,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12506,NonBGC_2501_1,"[Cluster_0992319, Cluster_0120706, Cluster_008...",Non,No,"[Cluster_1351173, EC:2.7.1.33, EC:1.6.5.11, Cl...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12507,NonBGC_2501_2,"[Cluster_0992319, Cluster_0120706, Cluster_008...",Non,No,"[Cluster_0576606, Cluster_0980814, Cluster_185...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12508,NonBGC_2501_3,"[Cluster_0992319, Cluster_0120706, Cluster_008...",Non,No,"[Cluster_1106154, Cluster_0451518, EC:4.3.3.7,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [231]:
BGC_TD_dataset_negative['length'] = BGC_TD_dataset_negative['sentence'].apply(len)
BGC_TD_dataset_negative['TDlength'] = BGC_TD_dataset_negative['TDsentence'].apply(len)
BGC_TD_dataset_negative['TDlabels'] = [[0]*128]*len(BGC_TD_dataset_negative)
BGC_TD_dataset_negative

,ID,sentence,labels,isBGC,TDsentence,TDlabels,length,TDlength
0,NonBGC_0_0,"[Cluster_1645774, Cluster_1021965, Cluster_186...",Non,No,"[Cluster_0510929, EC:1.1.1.22, EC:2.7.7.19, Cl...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",20,128
1,NonBGC_0_1,"[Cluster_1645774, Cluster_1021965, Cluster_186...",Non,No,"[Cluster_1037973, EC:3.6.3.25, EC:3.2.1.177, C...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",20,128
2,NonBGC_0_2,"[Cluster_1645774, Cluster_1021965, Cluster_186...",Non,No,"[Cluster_0364863, EC:2.7.1.180, Cluster_034784...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",20,128
3,NonBGC_0_3,"[Cluster_1645774, Cluster_1021965, Cluster_186...",Non,No,"[Cluster_0525603, EC:1.1.1.85, Cluster_1645774...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",20,128
4,NonBGC_0_4,"[Cluster_1645774, Cluster_1021965, Cluster_186...",Non,No,"[EC:1.2.4.2, Cluster_0945303, Cluster_1541072,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",20,128
...,...,...,...,...,...,...,...,...
12505,NonBGC_2501_0,"[Cluster_0992319, Cluster_0120706, Cluster_008...",Non,No,"[EC:3.5.2.2, Cluster_0279200, Cluster_1308801,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8,128
12506,NonBGC_2501_1,"[Cluster_0992319, Cluster_0120706, Cluster_008...",Non,No,"[Cluster_1351173, EC:2.7.1.33, EC:1.6.5.11, Cl...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8,128
12507,NonBGC_2501_2,"[Cluster_0992319, Cluster_0120706, Cluster_008...",Non,No,"[Cluster_0576606, Cluster_0980814, Cluster_185...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8,128
12508,NonBGC_2501_3,"[Cluster_0992319, Cluster_0120706, Cluster_008...",Non,No,"[Cluster_1106154, Cluster_0451518, EC:4.3.3.7,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8,128


In [232]:
BGC_TD_dataset_negative['sentence'] = BGC_TD_dataset_negative['sentence'].apply(lambda x:' '.join(x))
BGC_TD_dataset_negative['TDsentence'] = BGC_TD_dataset_negative['TDsentence'].apply(lambda x:' '.join(x))
BGC_TD_dataset_negative['TDlabels'] = BGC_TD_dataset_negative['TDlabels'].apply(lambda x:[str(i) for i in x]).apply(lambda x:' '.join(x))
BGC_TD_dataset_negative

,ID,sentence,labels,isBGC,TDsentence,TDlabels,length,TDlength
0,NonBGC_0_0,Cluster_1645774 Cluster_1021965 Cluster_186305...,Non,No,Cluster_0510929 EC:1.1.1.22 EC:2.7.7.19 Cluste...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,20,128
1,NonBGC_0_1,Cluster_1645774 Cluster_1021965 Cluster_186305...,Non,No,Cluster_1037973 EC:3.6.3.25 EC:3.2.1.177 Clust...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,20,128
2,NonBGC_0_2,Cluster_1645774 Cluster_1021965 Cluster_186305...,Non,No,Cluster_0364863 EC:2.7.1.180 Cluster_0347848 C...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,20,128
3,NonBGC_0_3,Cluster_1645774 Cluster_1021965 Cluster_186305...,Non,No,Cluster_0525603 EC:1.1.1.85 Cluster_1645774 Cl...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,20,128
4,NonBGC_0_4,Cluster_1645774 Cluster_1021965 Cluster_186305...,Non,No,EC:1.2.4.2 Cluster_0945303 Cluster_1541072 Clu...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,20,128
...,...,...,...,...,...,...,...,...
12505,NonBGC_2501_0,Cluster_0992319 Cluster_0120706 Cluster_008385...,Non,No,EC:3.5.2.2 Cluster_0279200 Cluster_1308801 Clu...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,8,128
12506,NonBGC_2501_1,Cluster_0992319 Cluster_0120706 Cluster_008385...,Non,No,Cluster_1351173 EC:2.7.1.33 EC:1.6.5.11 Cluste...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,8,128
12507,NonBGC_2501_2,Cluster_0992319 Cluster_0120706 Cluster_008385...,Non,No,Cluster_0576606 Cluster_0980814 Cluster_185434...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,8,128
12508,NonBGC_2501_3,Cluster_0992319 Cluster_0120706 Cluster_008385...,Non,No,Cluster_1106154 Cluster_0451518 EC:4.3.3.7 Clu...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,8,128


In [233]:
BGC_TD_dataset_negative.to_csv('../data/BGC_TD_dataset_negative.csv')

In [234]:
BGC_TD_dataset_positive

,ID,sentence,labels,isBGC,TDsentence,TDlabels,length,TDlength
0,BGC0000436_0,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,EC:3.1.1.23 EC:1.2.1.12 Cluster_0826651 Cluste...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,7,128
1,BGC0000436_1,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_0121728 Cluster_1707520 Cluster_068090...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,7,128
2,BGC0000436_2,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_1153963 Cluster_1501678 Cluster_160734...,1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,7,128
3,BGC0000436_3,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_0636113 EC:4.1.1.17 Cluster_1359171 EC...,0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 ...,7,128
4,BGC0000436_4,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_0898720 Cluster_0948693 Cluster_150030...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,7,128
...,...,...,...,...,...,...,...,...
12505,BGC0001595_0,Cluster_0923528 Cluster_1221717 Cluster_030676...,Alkaloid,Yes,EC:2.7.4.16 Cluster_0087115 Cluster_0059320 Cl...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,23,128
12506,BGC0001595_1,Cluster_0923528 Cluster_1221717 Cluster_030676...,Alkaloid,Yes,EC:3.5.3.18 Cluster_0715334 Cluster_0133357 EC...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,23,128
12507,BGC0001595_2,Cluster_0923528 Cluster_1221717 Cluster_030676...,Alkaloid,Yes,Cluster_1747296 Cluster_0169359 Cluster_036396...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,23,128
12508,BGC0001595_3,Cluster_0923528 Cluster_1221717 Cluster_030676...,Alkaloid,Yes,EC:3.5.1.88 Cluster_0329918 EC:1.2.4.2 Cluster...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,23,128


In [235]:
BGC_TD_dataset = BGC_TD_dataset_positive
BGC_TD_dataset = BGC_TD_dataset.append(BGC_TD_dataset_negative, ignore_index=True)

C:\Users\laiqi\AppData\Local\Temp\ipykernel_24656\788328818.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  BGC_TD_dataset = BGC_TD_dataset.append(BGC_TD_dataset_negative, ignore_index=True)


In [238]:
BGC_TD_dataset.to_csv('../data/BGC_TD_dataset.csv')

In [240]:
BGC_TD_dataset['labels']

0        NRP
1        NRP
2        NRP
3        NRP
4        NRP
        ... 
25015    Non
25016    Non
25017    Non
25018    Non
25019    Non
Name: labels, Length: 25020, dtype: object

In [248]:
len(BGC_TD_dataset.iloc[1679]['TDsentence'].split())

128